# Heroes Of Pymoli Data Analysis
- A significant majority of the players are Male(81%).
- Female players, on average, spend as much as Male players on in-game purchases
- A major chunk of players fall between the age group of 15-30 with a significant portion of them between the ages 20-24, implying young adults are more inclined to spend more on in-game purchases
- Since this data is only purchase data and not actual active player data, the above assumptions might be skewed towards the "purchasing" crowd than the non-purchasing crowd who might be enjoying the game without spending on in-game purchases

In [1]:
import pandas as pd

In [2]:
purchasefile = "purchase_data.json"
purchase_df = pd.read_json(purchasefile)

# Player Count

In [3]:
unique_users = purchase_df["SN"].value_counts()
totalplayers = pd.DataFrame({"Total Players": unique_users.count()},index = [0])
totalplayers

,Total Players
0,573


In [4]:
unique_items = purchase_df["Item ID"].value_counts()

In [5]:
purchase_analysis_total = pd.DataFrame ({"Number of Unique Items": unique_items.count(),
                                  "Average Price": purchase_df["Price"].mean(),
                                  "Number of Purchases": purchase_df["Item ID"].count(),
                                  "Total Revenue": purchase_df["Price"].sum()
                                  },index = [0])

In [6]:
purchase_analysis_total_reorg = purchase_analysis_total[["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]]
purchase_analysis_total_reorg["Average Price"] = purchase_analysis_total_reorg["Average Price"].map("${:,.2f}".format)
purchase_analysis_total_reorg["Total Revenue"] = purchase_analysis_total_reorg["Total Revenue"].map("${:,.2f}".format)

# Purchasing Analysis (Total)

In [7]:
purchase_analysis_total_reorg

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [8]:
female_df = purchase_df.loc[purchase_df["Gender"] == "Female",:]
uniquefemale = female_df["SN"].value_counts()

In [9]:
male_df = purchase_df.loc[purchase_df["Gender"] == "Male",:]
uniquemale = male_df["SN"].value_counts()

In [10]:
othergender_df = purchase_df.loc[(purchase_df["Gender"] != "Male") & (purchase_df["Gender"] != "Female"),:]
uniqueother = othergender_df["SN"].value_counts()

In [11]:
gender_grp = purchase_df.groupby(["Gender"], as_index = False)
genders =(gender_grp.mean())["Gender"]

In [12]:
gender_demo = pd.DataFrame({
    "Gender": genders,
    "Total Count": [uniquefemale.count(), uniquemale.count(), uniqueother.count()],
    "Percentage of Players":[round((uniquefemale.count()/unique_users.count())*100,2),round((uniquemale.count()/unique_users.count())*100,2),round((uniqueother.count()/unique_users.count())*100,2)]
    })

In [13]:
gender_sorted = gender_demo.sort_values("Total Count", ascending=False)
gender_sorted = gender_sorted.set_index("Gender")
gender_sorted.index.name = None

# Gender Demographics

In [14]:
gender_sorted

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [15]:
purchase_analysis_gender = pd.DataFrame({"Gender": genders,
                                        "Purchase count":(gender_grp.count())["Item ID"],
                                        "Average Purchase Price":(gender_grp.mean())["Price"],
                                        "Total Purchase Value":(gender_grp.sum())["Price"],
                                        "Normalized Totals":round((gender_grp.sum())["Price"]/gender_demo["Total Count"],2)
                                        })

In [16]:
purchase_analysis_gender_reorg = purchase_analysis_gender[["Gender","Purchase count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]
purchase_analysis_gender_reorg["Normalized Totals"] = purchase_analysis_gender_reorg["Normalized Totals"].map("${:,.2f}".format)
purchase_analysis_gender_reorg["Total Purchase Value"] = purchase_analysis_gender_reorg["Total Purchase Value"].map("${:,.2f}".format)
purchase_analysis_gender_reorg["Average Purchase Price"] = purchase_analysis_gender_reorg["Average Purchase Price"].map("${:,.2f}".format)

# Purchasing Analysis (Gender)

In [17]:
purchase_analysis_gender_reorg.set_index("Gender")

,Purchase count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [18]:
bins = [5,9,14,19,24,29,34,39,50]

# Create labels for these bins
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

Agedemo_series = pd.cut(purchase_df["Age"], bins, labels=group_labels)

In [19]:
purchase_df["AgeRange"] = Agedemo_series
purchase_df1 = purchase_df.drop_duplicates(['SN'], keep='first')

#grouped using unique players
Agedemo_grp = purchase_df1.groupby("AgeRange",as_index=False)

#grouped using total purchases which include multiple transactions by single player
Agedemo_grp1 = purchase_df.groupby("AgeRange",as_index=False)

# Age Demographics

In [20]:
age_demo = pd.DataFrame({" ":(Agedemo_grp.count())["AgeRange"],
                        "Percentage of Players":round(((Agedemo_grp.count())["SN"]/unique_users.count())*100,2),
                        "Total Count":(Agedemo_grp.count())["SN"]})
age_demo.set_index(" ")

,Percentage of Players,Total Count
,,
<10,3.32,19
10-14,4.01,23
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-39,4.71,27
40+,1.92,11


In [21]:
purchase_analysis_age = pd.DataFrame({" ":(Agedemo_grp.count())["AgeRange"],
                          "Purchase Count":(Agedemo_grp1.count())["SN"],
                          "Average Purchase Price":round((Agedemo_grp1.mean())["Price"],2),
                          "Total Purchase Value":(Agedemo_grp1.sum())["Price"],
                          "Normalized Totals":round(((Agedemo_grp1.sum())["Price"]/(Agedemo_grp.count())["SN"]),2)
                         })

In [22]:
purchase_analysis_age_reorg = purchase_analysis_age[[" ","Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]].set_index(" ")
purchase_analysis_age_reorg["Average Purchase Price"] = purchase_analysis_age_reorg["Average Purchase Price"].map("${:,.2f}".format)
purchase_analysis_age_reorg["Total Purchase Value"] = purchase_analysis_age_reorg["Total Purchase Value"].map("${:,.2f}".format)
purchase_analysis_age_reorg["Normalized Totals"] = purchase_analysis_age_reorg["Normalized Totals"].map("${:,.2f}".format)

# Purchasing Analysis (Age)

In [23]:
purchase_analysis_age_reorg

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


In [24]:
topspenders = purchase_df.groupby("SN")

In [25]:
topspendertotal = topspenders.sum()
topspendertotal["Average Purchase Price"] = (topspenders.mean())["Price"]
topspendertotal["Purchase Count"] = (topspenders.count())["Item ID"]
topspendertotal = topspendertotal.rename(columns={"Price": "Total Purchase Value"})

In [26]:
topspenders_sorted = topspendertotal.sort_values("Total Purchase Value", ascending = False)[["Purchase Count","Average Purchase Price","Total Purchase Value"]]
topspenders_sorted["Average Purchase Price"] = topspenders_sorted["Average Purchase Price"].map("${:,.2f}".format)
topspenders_sorted["Total Purchase Value"] = topspenders_sorted["Total Purchase Value"].map("${:,.2f}".format)

# Top Spenders

In [27]:
topspenders_sorted.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [28]:
mostpopular = purchase_df.groupby(["Item ID","Item Name"])

In [29]:
purchase_count = mostpopular.count()
purchase_count["Item Price"] = (mostpopular.mean())["Price"]
purchase_count["Total Purchase Value"] = (mostpopular.sum())["Price"]
purchase_count["Item Price"] = purchase_count["Item Price"].map("${:,.2f}".format)

In [30]:
purchase_count_popular = purchase_count[["SN","Item Price","Total Purchase Value"]].sort_values("SN",ascending = False).head(5)
purchase_count_popular = purchase_count_popular.rename(columns={"SN": "Purchase Count"})
purchase_count_popular["Total Purchase Value"] = purchase_count_popular["Total Purchase Value"].map("${:,.2f}".format)

# Most Popular Items

In [31]:
purchase_count_popular

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [32]:
purchase_count_profit = purchase_count[["SN","Item Price","Total Purchase Value"]].sort_values("Total Purchase Value",ascending = False).head(5)
purchase_count_profit = purchase_count_profit.rename(columns={"SN": "Purchase Count"})
purchase_count_profit["Total Purchase Value"] = purchase_count_profit["Total Purchase Value"].map("${:,.2f}".format)

# Most Profitable Items

In [33]:
purchase_count_profit

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
